# Gemini

In [3]:
from utils.model_setup import *
from utils.dataset_setup import *
from utils.experiment_setup import *
from utils.save_results import *
from utils.state_control import *
from utils.run_experiment import *

In [4]:
from importlib import reload

In [5]:
model_identifier = 'gemini'
path_to = "./data/"
api_key = 4 # With value -1 it will change every 10 iterations
execution_beggining = 1

## Setup

In [6]:
"""
Setup State
"""
use_experiment_state = False

state = create_state(-1, -1, -1, 1, 1, 0)
if use_experiment_state:
  state = retrieve_state(path_to, model_identifier)

In [7]:
x = input()

 tes


In [ ]:
execute_experiment(state, model_identifier, path_to, api_key%5 + 1, execution_beggining)

ATENTION - Choose the model:
1 - gemini-1.5-flash
2 - llama-3.2-90b-text-preview
3 - other


Input the number you would like to execute:  1


Choosen Model: gemini-1.5-flash


 Setting model gemini-1.5-flash with API Key 5 


genai.GenerativeModel(
    model_name='models/gemini-1.5-flash',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)
ATENCAO - Escolha o dataset:
1 - essaysFullGrade
2 - aes_enem_dataset
3 - propor2024
4 - extended2024
5 - extended_complete
6 - Outro


Coloque o número do dataset que gostaria de executar:  3


Dataset escolhido: propor2024
Propor2024 train size = 744
Propor2024 train and validation size = 960
Propor2024 total size = 1155
ATENCAO - Escolha o Experimento:
1 - experimento1
2 - experimento2
3 - experimento3
4 - experimento4
5 - experimento5
6 - experimento6
7 - Outro


Coloque o número do experimento que gostaria de executar:  5


Experimento escolhido: exp5
Create exp5
Prompt template: 
Qual o método de avaliação utilizado na correção de redações do ENEM?

####

Corrija a redação abaixo seguindo o método de avaliação do ENEM. Atribuindo notas quantitativas entre 0 e 200 para cada uma das cinco competências. Forneça uma explicação detalhada que justifique cada nota.

Tema:
"""
<TEMA>
"""


Textos de suporte:
"""
<SUPORTE> 
"""


Redação:
"""
<REDAÇÃO>
"""

####
Não tenho dados adicionais que possam contribuir, use apenas a sua própria base de dados para fornecer as notas seguindo o formato do ENEM.

Com os dados da reposta anterior, forneça apenas as notas quantitativas entre 0 e 200 para cada competência, e a nota final, no seguinte formato:[C1,C2,C3,C4,C5,Total], onde "Total" é a soma das notas das cinco competências. 

Não forneça texto na resposta, apenas as notas numéricas para competência e o total.
{'Model': -1, 'Database': -1, 'Experimento': -1, 'Execucao': 1, 'MaxExecucao': 1, 'Batch': 0}
{'Model': 1, '